In [1]:
import numpy as np
import scipy as sp
from scipy.stats import unitary_group

import cirq
from cirq import ops
from cirq.ops import CZ, H, CNOT, X, Y, Z, RotYGate, MeasurementGate
from cirq.google import XmonQubit, ExpWGate, ExpZGate, Exp11Gate, XmonMeasurementGate
from cirq.google import Simulator, ConvertToXmonGates, MergeRotations, MergeInteractions, EjectZ

In [2]:
import pickle

def saveCircuit(fName):
    with open(fName, 'wb') as f:
        pickle.dump(circuit, f)
def loadCircuit(fName):
    with open(fName, 'rb') as f:
        return pickle.load(f)

In [3]:
class CircuitCost(object):
    def global_cost(self, circuit):
        raise NotImplementedError
    def local_cost(self, subcircuit):
        raise NotImplementedError
    def cost(self, circuit):
        return self.global_cost(circuit) + self.local_cost(circuit)

class GateCountCost(CircuitCost):
    def global_cost(self, circuit):
        return 0
    def local_cost(self, circuit):
        #return sum((1 for moment in circuit for op in moment.operations))
        return sum((1 for op in circuit.iter_ops()))
    
class MultiQubitGateCost(CircuitCost):
    def global_cost(self, circuit):
        return 0
    def local_cost(self, circuit):
        #return sum((1 for moment in circuit for op in moment.operations if len(op.qubits) >= 2))
        return sum((1 for op in circuit.iter_ops() if len(op.qubits) >= 2))

In [4]:
class WildcardSymbol(cirq.Symbol):
    """A Symbol that tests equal to any regular value or symbol"""
    #def __init__(self):
    def __eq__(self, other):
        if isinstance(other, WildcardSymbol):
            return self is other
        else:
            return True

    def __neq__(self, other):
        return not self == other

class PlaceholderQubit(cirq.QubitId):
    """A placeholder for some particular qubit."""
    pass

#class OperationPattern(object):
#    """A description of an operator for some GatePattern."""
#    pass

class GatePattern(cirq.Gate):
    """A description of a gate with optional unspecified argument values."""
    def __init__(gate_type, *gate_args, **gate_kwargs) -> None:
        """Pass WildcardSymbol objects to each unspecified argument."""
        self.gate_type = gate_type
        self.gate_args = gate_args
        self.gate_kwargs = gate_kwargs
    #def on(self, *qubits: cirq.QubitId) -> cirq.Operation:
    #def validate_args(self, qubits: Sequence[QubitId]) -> None:
    def matches(self, gate, tolerance=0.):
        if tolerance != 0:
            raise NotImplementedError
        return self == gate
        

class CircuitPattern(object):
    def __init__(self, gate_patterns):
        self.gate_patterns = tuple(gate_patterns)
    def matches(self, gates, tolerance=0.):
        if tolerance != 0:
            raise NotImplementedError
        if len(gates) != len(self.gate_patterns):
            return False
        return all((gp.matches(gate) for gp, gate in zip(self.gate_patterns, gates)))


class MovementRule(object):
    gate_a_match = NotImplemented
    gate_b_match = NotImplemented
    def can_apply(self, gate_a, gate_b):
        pass
    def swapped(self, gate_a, gate_b):
        raise NotImplementedError
    def unswapped(self, gate_a, gate_b):
        ...

In [5]:
a = cirq.NamedQubit('a')
b = cirq.NamedQubit('b')

In [6]:
def make_inefficient_circuit(t=0.1, v=0.11):
    return cirq.Circuit.from_ops(
    #    H(b),
    #    CNOT(a, b),
    #    H(b),
    #    CNOT(a, b),
    #    CNOT(b, a),
    #    H(a),
    #    CNOT(a, b),
    #    Z(a)**t, Z(b)**-t,
    #    CNOT(a, b),
    #    H(a), Z(b)**v,
    #    CNOT(a, b),
    #    Z(a)**-v, Z(b)**-v,
        Y(b)**.5, #H(b),
        #CZ(a, b),
        #H(a), #H(b),
        CNOT(a, b),
        Y(b)**-.5,
    )

In [7]:
circuit = make_inefficient_circuit()
circuit.to_unitary_matrix().round(3)

array([[ 1.+0.j,  0.+0.j,  0.+0.j,  0.+0.j],
       [ 0.+0.j,  1.+0.j,  0.+0.j,  0.+0.j],
       [ 0.+0.j,  0.+0.j,  1.+0.j,  0.+0.j],
       [ 0.+0.j,  0.+0.j,  0.+0.j, -1.+0.j]])

In [8]:
GateCountCost().cost(circuit)

AttributeError: 'Circuit' object has no attribute 'iter_ops'

In [9]:
MultiQubitGateCost().cost(circuit)

AttributeError: 'Circuit' object has no attribute 'iter_ops'

In [348]:
ConvertToXmonGates().optimize_circuit(circuit)
MergeRotations().optimize_circuit(circuit)
ConvertToXmonGates().optimize_circuit(circuit)

In [10]:
print(circuit.to_text_diagram(transpose=True))

a b
│ │
│ Y^0.5
│ │
@─X
│ │
│ Y^-0.5
│ │



In [69]:
c2 = circuit[1:3] + circuit[-4:]
MergeRotations().optimize_circuit(c2)
print(c2)

a: ───────W(0.375)───

b: ───X──────────────


In [11]:
circuit

a───────@────────
        │
b─Y^0.5─X─Y^-0.5─

In [126]:
mr = unitary_group.rvs(2)
m1 = np.eye(2)
m0 = np.zeros((2,2))
m = np.concatenate([
        np.concatenate([m1, m0], 1),
        np.concatenate([m0, mr], 1)])

g, (a1, a0), (x, y, z), (b1, b0) = cirq.linalg.kak_decomposition(m)

q0 = cirq.NamedQubit('q0')
q1 = cirq.NamedQubit('q1')
circuit = cirq.Circuit.from_ops(
    cirq.google.decompositions.two_qubit_matrix_to_native_gates(q0, q1, m, allow_partial_czs=False))
MergeRotations().optimize_circuit(circuit)
print(circuit)

q0: ───W(0.406)^-0.5───Z^0.0937─────────────────────@───X^-0.283───@───Y^0.5─────────────Z^-0.134───
                                                    │              │
q1: ───────────────────W(0.886)^0.195───Z^-0.0937───Z──────────────Z───W(0.0755)^0.195───Z^-0.19────


In [119]:
x

0.5748610547900015

In [120]:
y

0.0

In [121]:
zmZ.dot(mH)

0.0

In [338]:
large_circ = loadCircuit('fermionSwap8.pkl')
large_circ2 = loadCircuit('fermionSplit8.pkl')

In [325]:
print(large_circ.to_text_diagram(transpose=True))

0         1         2         3         4         5         6         7         8
│         │         │         │         │         │         │         │         │
XXYY^1.91─XXYY      XXYY^0.0──XXYY      XXYY^1.91─XXYY      XXYY^1.91─XXYY      │
│         │         │         │         │         │         │         │         │
YXXY^0.0──#2        YXXY^0.0──#2        YXXY^0.0──#2        YXXY^0.0──#2        │
│         │         │         │         │         │         │         │         │
│         │         @^0.076───Z         │         │         │         │         │
│         │         │         │         │         │         │         │         │
×ᶠ────────×ᶠ        ×ᶠ────────×ᶠ        ×ᶠ────────×ᶠ        ×ᶠ────────×ᶠ        │
│         │         │         │         │         │         │         │         │
│         XXYY^1.91─XXYY      XXYY^1.91─XXYY      XXYY^1.91─XXYY      XXYY^1.91─XXYY
│         │         │         │         │         │         │         │         │
│         YXX

In [339]:
ConvertToXmonGates().optimize_circuit(large_circ)

In [340]:
ConvertToXmonGates().optimize_circuit(large_circ2)

In [314]:
MergeInteractions(allow_partial_czs=True).optimize_circuit(large_circ)

In [342]:
MergeInteractions(allow_partial_czs=True).optimize_circuit(large_circ2)

In [360]:
EjectZ(tolerance=1e-6).optimize_circuit(large_circ)

In [361]:
EjectZ(tolerance=1e-6).optimize_circuit(large_circ2)

In [362]:
MergeRotations().optimize_circuit(large_circ)

In [363]:
MergeRotations().optimize_circuit(large_circ2)

In [364]:
print(cirq.Circuit.from_ops(
    large_circ2[:200].iter_ops(),
    strategy=cirq.InsertStrategy.EARLIEST
).to_text_diagram(transpose=True))

1            2               3                4                5               6               7                8
│            │               │                │                │               │               │                │
W(0.721)^0.5 W(0.372)^-0.5   W(0.618)^-0.5    W(0.893)^0.5     W(0.826)^0.5    W(0.238)^-0.5   W(0.472)^-0.5    W(0.972)^-0.5
│            │               │                │                │               │               │                │
Z^-0.221     Z^0.128         Z^0.882          Z^-0.393         Z^0.674         Z^0.262         Z^-0.972         Z^0.528
│            │               │                │                │               │               │                │
│            │               │                │                │               │               @^0.5────────────Z
│            │               │                │                │               │               │                │
│            │               │                │                │      

In [269]:
#diag = large_circ.to_text_diagram(transpose=True)
diag = cirq.Circuit.from_ops(large_circ.iter_ops(), strategy=cirq.InsertStrategy.EARLIEST).to_text_diagram(transpose=True)
with open('diagram.txt', 'w') as f:
    f.write(diag)

In [365]:
#diag = large_circ.to_text_diagram(transpose=True)
diag = cirq.Circuit.from_ops(large_circ2.iter_ops(), strategy=cirq.InsertStrategy.EARLIEST).to_text_diagram(transpose=True)
with open('diagram2b.txt', 'w') as f:
    f.write(diag)

In [267]:
len(cirq.Circuit.from_ops(large_circ.iter_ops(), strategy=cirq.InsertStrategy.EARLIEST))

127

In [257]:
len(cirq.Circuit.from_ops(large_circ.iter_ops(), strategy=cirq.InsertStrategy.EARLIEST))

135

In [295]:
len(cirq.Circuit.from_ops(large_circ2.iter_ops(), strategy=cirq.InsertStrategy.EARLIEST))

476

In [318]:
len(cirq.Circuit.from_ops(large_circ2.iter_ops(), strategy=cirq.InsertStrategy.EARLIEST))

431

In [149]:
MergeInteractions().optimize_circuit(large_circ)
MergeRotations().optimize_circuit(large_circ)